In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../../notebook_format')
from formats import load_style
load_style()

In [2]:
os.chdir(path)

# Data Structure

> Reimplementation from the [Python3 Cookbook Chapter 1. Data Structures and Algorithms](http://chimera.labs.oreilly.com/books/1230000000393/ch01.html)

## Simple Assignments to Unpack Iterables into Separate Variables

**Example1:** Unpacking a tuple.

In [3]:
p = (4, 5)
x, y = p
print(x)
print(y)

4
5


This works for all types of sequences (iterables), including tuples, list, strings, generators, files.

**Example2:** If you want to discard some of the elements, simply use `_` to represent it (You can use anything you want, this is just convention).

In [4]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
_, shares, price, _ = data
print(shares)
print(price)

50
91.1


## Use "Star Expressions" to Unpack Iterables of Arbitrary Length

**Example1:** Data that has name, e-mail and arbitrary number of telephone numbers.

In [5]:
record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = record
phone_numbers

['773-555-1212', '847-555-1212']

The star expression will always unpack a list (including none).

**Example2:** Performing different actions when looping over different "tagged" tuples. "Tagged" simply means they have some known pattern or structure. e.g. The first element is a tag indicating what other information does this tag contain.

In [6]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]

def do_foo( x, y ):
    print( 'foo', x, y )
    
def do_bar(s):
    print( 'bar', s )

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


**Example3:** String manipulation and throwing away variables.

In [7]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *_, homedir, sh = line.split(':')
print(uname)
print(homedir)
print(sh)

nobody
/var/empty
/usr/bin/false


## Keeping the Last N Items Using deque

**Example1:** A fix-sized queue that removes the oldest item when a new item is added and the queue is full.

In [8]:
from collections import deque

# specify the maxlen argument
q = deque( maxlen = 3 )
q.append(1)
q.append(2)
q.append(3)
print(q)
q.append(4)
print(q)

deque([1, 2, 3], maxlen=3)
deque([2, 3, 4], maxlen=3)


**Example2:** A unbounded queue. You can pop and add item from both end with O(1).

In [9]:
q = deque()
q.append(1)
q.append(2)
q.append(3)
print(q)

q.appendleft(4)
print(q)

# removes the right-most element
print( q.pop() )
print(q)

# removes the left-most element
print( q.popleft() )
print(q)

deque([1, 2, 3])
deque([4, 1, 2, 3])
3
deque([4, 1, 2])
4
deque([1, 2])


## Finding the Largest or Smallest N Items Using heapq

**Example1:** `nlargest()` and `nsmallest()`.

In [10]:
import heapq

nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print( heapq.nlargest( 3, nums ) )
print( heapq.nsmallest( 3, nums ) )

[42, 37, 23]
[-4, 1, 2]


**Example2:** `nlargest()` and `nsmallest()` with more complex data structure.

In [11]:
portfolio = [
   {'name': 'IBM', 'shares': 100, 'price': 91.1},
   {'name': 'AAPL', 'shares': 50, 'price': 543.22},
   {'name': 'FB', 'shares': 200, 'price': 21.09},
   {'name': 'HPQ', 'shares': 35, 'price': 31.75},
   {'name': 'YHOO', 'shares': 45, 'price': 16.35},
   {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

cheap = heapq.nsmallest( 3, portfolio, key = lambda s: s['price'] )
cheap

[{'name': 'YHOO', 'price': 16.35, 'shares': 45},
 {'name': 'FB', 'price': 21.09, 'shares': 200},
 {'name': 'HPQ', 'price': 31.75, 'shares': 35}]

When to use what:

- Use `nlargest()` and `nsmallest()` if you are trying to find a relatively small number of items. 
- Use `min()` and `max()` if you are simply trying to find the single smallest or largest item (N=1). 
- Use `sorted(items)[:N]` or `sorted(items)[-N:]` if N is about the same size as the input.

## PriorityQueue

A queue that sorts items by a given priority and always returns the item with the highest priority on each pop operation.

In [12]:
import heapq

class PriorityQueue(object):
    
    def __init__(self):
        self._queue = []
        self._index = 0
    
    def push( self, item, priority ):
        # the ( -priority, self._index, item ) is explained below
        heapq.heappush( self._queue, ( -priority, self._index, item ) )
        self._index += 1
    
    def pop(self):
        # [-1] just reutrns the item
        return heapq.heappop(self._queue)[-1]


class Item(object):
    
    def __init__( self, name ):
        self.name = name
    
    def __repr__(self):
        #  !r - convert the value to a string using repr().
        return 'Item({!r})'.format(self.name)    

In [13]:
pq = PriorityQueue()
pq.push( Item('foo'), 1 )
pq.push( Item('bar'), 5 )
pq.push( Item('spam'), 4 )
pq.push( Item('grok'), 1 )
print( pq.pop() )
print( pq.pop() )

Item('bar')
Item('spam')


In [14]:
# 1. you can't compare instances of Item
# the code below will return an error
# a = Item('foo')
# b = Item('bar')
# a < b

# 2. If you make (priority, item) tuples, they can be compared as 
# long as the priorities are different.
a = ( 1, Item('foo') )
b = ( 5, Item('bar') )
c = ( 1, Item('grok') )
print( a < b )
# print( a < c ) # returns an error since a and c are equa


True


Introducing the extra index and making (priority, index, item) tuples, you avoid this problem entirely, since no two tuples will ever have the same value for index (and Python never bothers to compare the remaining tuple values once the result of comparison can be determined).

## Keeping Dictionaries in Order Using OrderedDict

In [15]:
from collections import OrderedDict

d = dict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4

for key in d:
    print(key, d[key], end = "; ")

print()
od = OrderedDict()
od['foo'] = 1
od['bar'] = 2
od['spam'] = 3
od['grok'] = 4

# the order remains the same compared to the dict
for key in od:
    print(key, od[key], end = "; ")

grok 4; bar 2; spam 3; foo 1; 
foo 1; bar 2; spam 3; grok 4; 

Be aware that the size of an **OrderedDict** is more than twice as large as a normal dictionary!!

In [16]:
# one useful case may be keeping the order when converting it into json
import json
json.dumps(od)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

## Calculating with Dictionaries Using zip

Performing calculations (e.g., min, max, sorting, etc.) on a dictionary by convert the it into tuples of ( value, key ).

In [17]:
# company and stocks
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}

min_price = min( zip( prices.values(), prices.keys() ) )
print(min_price)

(10.75, 'FB')


## Finding Commonalities in Two Dictionaries Using set operations

You can perform common set operations with dictionary `.keys()` and `.items()` without having to convert them into a set, as they are unique.

In [18]:
a = {
   'x' : 1,
   'y' : 2,
   'z' : 3
}

b = {
   'w' : 10,
   'x' : 11,
   'y' : 2
}

In [19]:
# common keys
print( a.keys() & b.keys() )

# keys in a that are not in b
print( a.keys() - b.keys() )

# (key,value) pairs in common
print( a.items() & b.items() )

# make a new dictionary with certain keys removed
c = { key: a[key] for key in a.keys() - {'z'} }
print(c)

{'y', 'x'}
{'z'}
{('y', 2)}
{'y': 2, 'x': 1}


## Removing Duplicates from a Sequence while Maintaining Order

In [20]:
# simply calling set() removes duplicated but does not retain the original order
a = [ 1, 5, 2, 1, 9, 1, 5, 10 ]
print( set(a) )

{1, 2, 10, 5, 9}


In [21]:
def dedupe(items):
    seen = set()
    
    for item in items:
        if item not in seen:
            seen.add(item)
            yield item

list( dedupe(a) )

[1, 5, 2, 9, 10]

## Naming Slices

In [22]:
items = [ 0, 1, 2, 3, 4, 5, 6 ]
important = slice( 2, 4, 1 )

# instead of hardcoding the indexing slicing
# we name the slice so we will know what is means
# when we look at it later
print( items[2:4] )
print( items[important] )

# you can also look at various info of the slice
print( important.start )
print( important.stop )
print( important.step )

[2, 3]
[2, 3]
2
4
1


## Find the Most Frequently Items in a Sequence Using Counter

The `most_common` functionality from **Counter**.

In [23]:
from collections import Counter

words = [
   'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
   'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
   'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
   'my', 'eyes', "you're", 'under'
]
word_counts = Counter(words)
top_three = word_counts.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


**Counter** is simply a dictionary and we can manually increment the count.

In [24]:
morewords = ['why','are','you','not','looking','in','my','eyes']
for word in morewords:
    word_counts[word] += 1

word_counts['eyes']

9

In [25]:
# we can also perform math operations between them
a = Counter(words)
b = Counter(morewords)
print( a + b )

Counter({'eyes': 9, 'the': 5, 'my': 4, 'look': 4, 'into': 3, 'not': 2, 'around': 2, 'in': 1, "don't": 1, 'looking': 1, 'you': 1, 'are': 1, "you're": 1, 'under': 1, 'why': 1})


## Sorting with itemgetter

In [26]:
from operator import itemgetter

rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}
]

rows_by_fname = sorted( rows, key = itemgetter('fname') )
rows_by_fname

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]

The method is faster then `key = lambda r: r['fname']`. And you can assign multiple values inside the **itemgetter()**.

## Sorting with attrgetter

In [27]:
from operator import attrgetter

class User(object):
    def __init__(self, user_id):
        self.user_id = user_id

    def __repr__(self):
        return 'User({})'.format(self.user_id)

users = [ User(23), User(3), User(99) ]

print( sorted( users, key = attrgetter('user_id'), reverse = True ) )
print( min( users, key = attrgetter('user_id') ) )

[User(99), User(23), User(3)]
User(3)


## Grouping Records Together Using groupby

**Example1:** The **groupby()** function works by scanning a sequence and finding sequential "runs" of identical values . On each iteration, it returns the value along with an iterator that produces all of the items in a group with the same value.

An important preliminary step is sorting the data according to the field of interest. Since groupby() only examines consecutive items, failing to sort first won’t group the records as you want.

In [28]:
from operator import itemgetter
from itertools import groupby

rows = [
    {'address': '5412 N CLARK', 'date': '07/01/2012'},
    {'address': '5148 N CLARK', 'date': '07/04/2012'},
    {'address': '5800 E 58TH', 'date': '07/02/2012'},
    {'address': '2122 N CLARK', 'date': '07/03/2012'},
    {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
    {'address': '1060 W ADDISON', 'date': '07/02/2012'},
    {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
    {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

# Sort by the desired field first
rows.sort(key = itemgetter('date'))

# Iterate in groups
for date, items in groupby( rows, key = itemgetter('date') ):
    print(date)
    for i in items:
        print('    ', i )

07/01/2012
     {'address': '5412 N CLARK', 'date': '07/01/2012'}
     {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
     {'address': '5800 E 58TH', 'date': '07/02/2012'}
     {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
     {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
     {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
     {'address': '5148 N CLARK', 'date': '07/04/2012'}
     {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


**Example2:** Use **defaultdict** to add every item if you do not care about the ordering.

In [29]:
from collections import defaultdict

rows_by_date = defaultdict(list)

for row in rows:
    rows_by_date[ row['date'] ].append(row)

for r in rows_by_date:
    print(r)
    for i in rows_by_date[r]:
        print('    ', i )

07/01/2012
     {'address': '5412 N CLARK', 'date': '07/01/2012'}
     {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/03/2012
     {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/02/2012
     {'address': '5800 E 58TH', 'date': '07/02/2012'}
     {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
     {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/04/2012
     {'address': '5148 N CLARK', 'date': '07/04/2012'}
     {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


## Filtering Sequence Elements

**Example1:** Replacing the values that don’t meet the criteria with a new value.

In [30]:
mylist = [1, 4, -5, 10, -7, 2, 3, -1]
clip_neg = [ m if m > 0 else 0 for m in mylist ]
clip_neg

[1, 4, 0, 10, 0, 2, 3, 0]

**Example2:** Use `filter` when the filtering criteria can't be easily expressed.

In [31]:
values = ['1', '2', '-3', '-', '4', 'N/A', '5']

def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False

# filter returns an iterator, you'll have to manually
# convert it to a list
ivals = list( filter( is_int, values ) )
print(ivals)

['1', '2', '-3', '4', '5']


**Example3:** `itertools.compress()` for filtering a sequence with an accompanying boolean sequence. Suppose you want to make a list of all addresses where the corresponding count value was greater than 5.

In [32]:
from itertools import compress

addresses = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK'
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
]

counts = [ 0, 3, 10, 4, 1, 7, 6, 1 ]
more5 = [ n > 5 for n in counts ]

# it also returns an  iterator
print( list( compress( addresses, more5 ) ) )

['5800 E 58TH', '4801 N BROADWAY', '1039 W GRANVILLE']


## Extracting a Subset of a Dictionary

In [33]:
prices = {
   'ACME': 45.23,
   'AAPL': 612.78,
   'IBM': 205.55,
   'HPQ': 37.20,
   'FB': 10.75
}

# Make a dictionary of all prices over 200
p1 = { key: value for key, value in prices.items() if value > 200 }

# Make a dictionary of tech stocks
tech_names = { 'AAPL', 'IBM', 'HPQ', 'MSFT' }
p2 = { key: value for key, value in prices.items() if key in tech_names }

## Mapping Names to Sequence Elements Using namedtuple

Be less dependent on position of the element in a tuple, by accessing the elements by name

In [34]:
from collections import namedtuple

Subscriber = namedtuple( 'Subscriber', ['addr', 'joined'] )
sub = Subscriber('jonesy@example.com', '2012-10-19')
sub.addr

'jonesy@example.com'

**namedtuple** can be used as a replacement for a dictionary, which requires more space to store.  However, be aware that a it's immutable.

In [35]:
Stock = namedtuple( 'Stock', ['name', 'shares', 'price'] )
s = Stock('ACME', 100, 123.45)

# s.shares = 75 # this will return an error
# use ._replace() if you really want to change the value
s = s._replace( shares = 75 )
s

Stock(name='ACME', shares=75, price=123.45)

## Transforming and Reducing Data Using Generation Expression

**Example1:** Notice that we do not need to write `sum( ( x * x for x in nums ) )` to first create the generator.

In [36]:
nums = [1, 2, 3, 4, 5]
s = sum( x * x for x in nums )
s

55

**Example2:** Determine if any .py files exist in a directory.

In [37]:
"""
import os

files = os.listdir('dirname')
if any( name.endswith('.py') for name in files ):
    print('There be python!')
else:
    print('Sorry, no python.')
"""
print("uncomment to run, but there's no directory named 'dirname'")

uncomment to run, but there's no directory named 'dirname'
